<a href="https://colab.research.google.com/github/CPukszta/BI-BE-CS-183-2023/blob/main/HW4/Problem2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bi/Be/Cs 183 2022-2023: Intro to Computational Biology
TAs: Meichen Fang, Tara Chari, Zitong (Jerry) Wang

**Submit your notebooks by sharing a clickable link with Viewer access. Link must be accessible from submitted assignment document.**

Make sure Runtime $\rightarrow$ Restart and run all works without error

**HW 4 Problem 2**

In this problem you will develop code for running the EM algorithm on a small toy example of transcripts, by iteratively estimating the read counts per transcript and approximating the relative transcript abundances from those counts.


##**Import data and install packages**

In [37]:
import numpy as np
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt #Can use other plotting packages like seaborn

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

## **Problem 2 (30 points)**
This contains parts b-d of for Problem 2. 

Below you can see an example of the first few steps of the EM procedure with this toy example. This begins with an initial, uniform guess for the Transcript abundances $\alpha$, which is then iteratively updated by running the expecation (E) and maximization (M) steps. We are assuming transcripts of the same length here.

<img src="https://drive.google.com/uc?export=view&id=1C6U4n4hNS7WCPzB8yNdSGb2rovuaf0M4" alt="EMFigure" width="500" height="500">

From the compatibility matrix $\mathbf{Y}$ we can see the alignment of the $N$ Reads to the $K$ Transcripts. For example, Read $c$ does not align to any sequence in Transcript $green$ (the value at Y[1,2] = 0).



In [2]:
#Compatibility Matrix for the diagram

Y = np.array([[1,0,1,1,1],[1,1,0,0,1],[1, 1, 1 ,0 ,0]])

YLabeled = pd.DataFrame(Y, index=['red','green','blue'], columns=['a','b','c','d','e'])

print(Y)
print(YLabeled)

[[1 0 1 1 1]
 [1 1 0 0 1]
 [1 1 1 0 0]]
       a  b  c  d  e
red    1  0  1  1  1
green  1  1  0  0  1
blue   1  1  1  0  0


Let $\alpha$ represent transcript abundance estimates ($\alpha_{red}, \alpha_{blue}, \alpha_{green} $), start at all $\alpha = 0.33$ (all transcripts equally represented/counted). 

The Q function for optimization is $\sum_{n=1}^N \sum_{k=1}^K \frac{ y_{k,n} \alpha^{(t)}_{k} }{ \sum_{l=1}^K  y_{ln} \alpha^{(t)}_{l}} \log ( y_{k,n} \alpha_{k})$.

### **b) Implement the expectation (E) step as a function (5 points)**

In the E step we will calculate the posterior \begin{align}
p(Z_n=k|Y_n;\alpha^{(t)}) = \frac{  y_{k,n} \alpha^{(t)}_{k}}{ \sum_{l=1}^K y_{l,n} \alpha^{(t)}_{l}}\end{align}
for each pair of the $n$ reads (columns) and $k$ transcripts (rows) given a set of transcript abundances $\alpha$.




**Fill in the e_step() function below to accept a given vector of transcript abundances ($\alpha^{(t)}$) and the compatibility matrix, and return the values of the posterior.**

In [3]:
#Implement expectation step as function given transcript abundance estimates alpha and the compatibility matrix Y
def e_step(Y,alpha):
  '''Takes in a compatability matrix Y and a 1x n vector of transcript abundances and returns the Expectation step of the EM'''
  num = Y*np.transpose(alpha)
  denom = np.sum(Y*np.transpose(alpha))
  return num/denom


In [22]:
a = np.array([[0.33,0.33,0.33]])
y_old = e_step(Y,a)
y_old

array([[0.1, 0. , 0.1, 0.1, 0.1],
       [0.1, 0.1, 0. , 0. , 0.1],
       [0.1, 0.1, 0.1, 0. , 0. ]])

### **c) Implement the maximization (M) step as a function (5 points)**

During the maximization step we re-calculate the values for $\alpha$, given the posterior values and the previously determined $\alpha^{(t)}$ values. The formula for the new $\alpha$ values which maximize the $Q$ function is
\begin{align}
 \alpha_k^{(t+1)}= \frac{1}{N}\sum_{n=1}^N  \frac{ y_{k,n} \alpha^{(t)}_{k}}{\sum_{l=1}^K  y_{l,n} \alpha^{(t)}_{l}} \quad.
\end{align}

**Fill in the m_step() function below to accept the posterior values and to return a vector of updated abundance estimates $\alpha$ for each transcript.**

In [32]:
#Implement maximization step as function taking in Read counts for each transcript
def m_step(y_old,alpha_old):
  '''Takes in the result of the E step and the old iteration for alpha and returns the next iteration for alpha'''
  N = np.shape(y_old)[1]
  num= y_old*np.transpose(alpha_old)
  denom = np.sum(y_old*np.transpose(alpha_old),axis=0)
  alpha = (1/N)*np.sum(num/denom,axis=1)
  return np.array(alpha)

In [33]:
m_step(y_old,a)

array([0.46666667, 0.26666667, 0.26666667])

### **d) Run the EM algorithm to produce estimated abundaces for $\alpha_{red}, \alpha_{blue}, \alpha_{green}$ and plot the log likelihood at each iteration. (10 points)**

Begin with $\alpha_{red} = \alpha_{blue} = \alpha_{green} = 0.33$, successively call the E then M functions until the $\alpha$ values from the current M step are within .0001 of the previous M step estimates.

**Implement the EM algorithm as described and (1) report the final $\alpha$ abundance values and (2) plot the log likelihood (not the Q-function) at each iteration.**

In [36]:
#Call both until < .0001 difference in estimated abundances
alpha_old = np.array([[0.33,0.33,0.33]])
it_diff = np.array([[1,1,1]])
Y_old = Y
while np.max(np.abs(it_diff))>0.001:
    Y_new = e_step(Y_old,alpha_old)
    alpha_new = m_step(Y_new,alpha_old)
    it_diff = alpha_new - alpha_old
    y_new = y_old
    alpha_old = [alpha_new]
    if np.max(np.abs(it_diff))<=0.001:
        break

print(alpha_new)

#need to plot log like at each iteration

[0.78543821 0.10728089 0.10728089]


In [ ]:
x=np.linspace((0,np.len(log_like_vals)))
p = bokeh.plotting.figure(
    width = 400, height = 500,
    title="yes"
)

p.circle(x,log_like_vals,color="blue")

bokeh.io.show(p)

### **e) Comment on the behavior of the likelihood plot i.e. monotonicity, plateaus, etc, and if it is expected behavior. (5 points)**